In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
%matplotlib inline
import csv

from functools import reduce

# 1. compare based on doi

## import data

In [2]:
#Import data
wos_doi = pd.read_pickle(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/first_clean/data_scholar_all_year_merge.pkl')[
                        ['UT (Unique WOS ID)','DOI']]
wos_doi.head()

,UT (Unique WOS ID),DOI
0,wos:a1975ak27200035,NaN
1,wos:a1975bb78600005,10.1007/bf00268754
2,wos:a1975ak49300025,10.1016/0042-6822(75)90198-1
3,wos:a1975al49000001,10.1016/0022-2836(75)90178-3
4,wos:a1975ay53300001,10.1016/0042-6822(75)90142-7


In [3]:
wos_doi.count().to_frame()

,0
UT (Unique WOS ID),61544
DOI,56796


In [4]:
wos_doi.columns

Index(['UT (Unique WOS ID)', 'DOI'], dtype='object')

In [5]:
# wos_doi.drop_duplicates(subset=['UT (Unique WOS ID)'])

In [6]:
## How many studies in WOS ?
wos_doi.drop_duplicates(inplace = True)
print(">>>", " total number of WOS publications : ", wos_doi.shape[0])

>>>  total number of WOS publications :  61544


In [7]:
## How many WOS studies have DOI  ?
wos_have_doi_have_doi = wos_doi[wos_doi['DOI'].notnull()]
print(">>>", " total number of WOS publications with DOI : ", wos_have_doi_have_doi.shape[0])

>>>  total number of WOS publications with DOI :  56796


In [8]:
## How many WOS studies have DOI  ?
wos_have_doi_no_doi = wos_doi[wos_doi['DOI'].isnull()]
print(">>>", " total number of WOS publications with no DOI : ", wos_have_doi_no_doi.shape[0])

>>>  total number of WOS publications with no DOI :  4748


In [9]:
wos_have_doi_no_doi.count()

UT (Unique WOS ID)    4748
DOI                      0
dtype: int64

## get Unpaywall data via the API

In [10]:
import os
myemail = 'dmbogning15@gmail.com'

In [11]:
import requests
import json
import time

def get_doi_data(doi):
    email = myemail or ''
    url = 'https://api.unpaywall.org/v2/{}?email={}'.format(doi,email)
    r = requests.get(url)
    data = r.json()
    response_time = r.elapsed.total_seconds()
    # simple rule for sleeping if responses are slow
    if response_time > 5.0:
        print(response_time, 'seconds for last request')
        print('Unpaywall slow to respond. Sleep for a few seconds.')
        time.sleep(int(response_time))
        print('Finished sleeping')
    return data

In [12]:
wos_doi_na = wos_doi.dropna()
list_wos_doi = list(wos_doi_na['DOI'])
list_wos_doi[:5]

['10.1007/bf00268754',
 '10.1016/0042-6822(75)90198-1',
 '10.1016/0022-2836(75)90178-3',
 '10.1016/0042-6822(75)90142-7',
 '10.1016/0022-2836(75)90132-1']

In [13]:
len(list_wos_doi)

56796

In [14]:
# Python code to split a list
# into sublists of given length.
#list_doi_all = list(databases_merged.reset_index()['DOI'])#[:26]
list_wos_doi_split = [list_wos_doi[x:x+1000] for x in range(0, len(list_wos_doi), 1000)]

In [15]:
len(list_wos_doi)

56796

In [16]:
range(len(list_wos_doi_split))

range(0, 57)

In [17]:
# list_wos_doi_split[0]

In [18]:
# data_upw = pd.DataFrame()
for elt in range(24,len(list_wos_doi_split)): # len(list_doi_split)  # range(21, 25)
    data_upw = []
    for doi in list_wos_doi_split[elt] :
        data_doi = get_doi_data(doi)
#         data_upw = pd.concat([data_upw,data_doi])
        data_upw.append(data_doi)
    
    data_upw = pd.json_normalize(data_upw)
    
    data_upw.to_csv(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/open_access_upw/data_upw_02072023_{elt}.csv')
#     data_upw.to_excel(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/data_upw_02072023.xls')
    data_upw.to_pickle(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/open_access_upw/data_upw_02072023_{elt}.pkl')
    print(f"#+++>>> total number of publication found in unpaywall for list_wos_doi_split_{elt} :", data_upw.shape[0]," #") 

#+++>>> total number of publication found in unpaywall for list_wos_doi_split_24 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_25 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_26 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_27 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_28 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_29 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_30 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_31 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_32 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_33 : 1000  #
#+++>>> total number of publication found in unpaywall for list_wos_doi_split_34 : 1000  #

In [19]:
vvv

NameError: name 'vvv' is not defined

In [ ]:
merged_df = pd.merge(wos_doi.reset_index(), data_upw.rename({'doi':'DOI'}, axis=1), on='DOI', how = 'left').drop('index', axis=1)
merged_df = merged_df.drop_duplicates(['UT (Unique WOS ID)'])

merged_df.to_csv(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/phage_data_upw_02072023.csv')
# merged_df.to_excel(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/phage_data_upw_02072023.xls')
merged_df.to_pickle(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/phage_data_upw_02072023.pkl')
merged_df

In [ ]:
len(wos_doi['DOI'].unique())

In [ ]:
len(merged_df['DOI'].unique())#.count()

In [ ]:
merged_df['DOI'].count()

In [ ]:
data_upw.columns

In [ ]:
data_upw['is_oa'].value_counts()

In [ ]:
data_upw['oa_status'].value_counts()